In [11]:
from janome.tokenizer import Tokenizer

# 開始、終端記号を設定する。
BEGIN = '__BEGIN__'
END = '__END__'
# 実際に一文を3単語ずつに分けてみる。
sentence = 'おいしいビールを飲もう'

t = Tokenizer()
# ここで型変換しないとgeneratorのリストができてしまう
# 分かち書きした文章に開始終端記号を追加してリストにする
words = list(t.tokenize(sentence, wakati=True))
words = [BEGIN] + words + [END]

# ['__BEGIN__', 'おいしい', 'ビール', 'を', '飲も', 'う', '__END__']
# 3単語ずつを1つのリストにすると全体の単語数ー２の回数回さないといけない
three_words_list = []
for i in range(len(words) - 2):
    #3単語ずつ一つリストに追加していく
    three_words_list.append(words[i:i + 3])
three_words_list

['__BEGIN__', 'おいしい', 'ビール', 'を', '飲も', 'う', '__END__']
おいしい	形容詞,自立,*,*,形容詞・イ段,基本形,おいしい,オイシイ,オイシイ
ビール	名詞,一般,*,*,*,*,ビール,ビール,ビール
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
飲も	動詞,自立,*,*,五段・マ行,未然ウ接続,飲む,ノモ,ノモ
う	助動詞,*,*,*,不変化型,基本形,う,ウ,ウ


[['__BEGIN__', 'おいしい', 'ビール'],
 ['おいしい', 'ビール', 'を'],
 ['ビール', 'を', '飲も'],
 ['を', '飲も', 'う'],
 ['飲も', 'う', '__END__']]

In [20]:
from collections import Counter
# さっきのをモジュール化する
def get_three_words_list(sentence):
    """文章を3単語の組にして返す"""
    t = Tokenizer()
    words = list(t.tokenize(sentence, wakati=True))
    words = [BEGIN] + words + [END]
    three_words_list = []
    for i in range(len(words) - 2):
        three_words_list.append(tuple(words[i:i+3]))
    return three_words_list

sentences = ['おいしいビールを飲もう', 'ビールを飲もう', 'おいしいビールは生']
three_words_list = []
# リストに追加されている3単語つづりの内容の出現率をカウントしてリストに追加
for sentence in sentences:
    three_words_list += get_three_words_list(sentence)
three_words_count = Counter(three_words_list)
three_words_count


Counter({('__BEGIN__', 'おいしい', 'ビール'): 2,
         ('おいしい', 'ビール', 'を'): 1,
         ('ビール', 'を', '飲も'): 2,
         ('を', '飲も', 'う'): 2,
         ('飲も', 'う', '__END__'): 2,
         ('__BEGIN__', 'ビール', 'を'): 1,
         ('おいしい', 'ビール', 'は'): 1,
         ('ビール', 'は', '生'): 1,
         ('は', '生', '__END__'): 1})

In [25]:
def generate_marcov_dict(three_words_count):
    """マルコフ連鎖での文章作成用の辞書データを作成する"""
    # key : 前半二つの単語
    # value : 次の単語とその重みとして出現回数を追加する
    markov_dict = {}
    #　前半の3単語とその重みで分ける
    #three_words = 3単語, count = 重み
    # for文の処理の中でさらに前半2単語後半1単語に分ける。
    for three_words, count in three_words_count.items():
        two_words = three_words[:2]
        next_word = three_words[2]
        # keyが新出の場合のみ辞書に追加
        if two_words not in markov_dict:
            markov_dict[two_words] = {'words': [], 'weights': []}
        # keyが既出でvalueが新出のときはここで追加されていく。
        markov_dict[two_words]['words'].append(next_word)
        markov_dict[two_words]['weights'].append(count)
    return markov_dict

markov_dict = generate_marcov_dict(three_words_count)
markov_dict


{('__BEGIN__', 'おいしい'): {'words': ['ビール'], 'weights': [2]},
 ('おいしい', 'ビール'): {'words': ['を', 'は'], 'weights': [1, 1]},
 ('ビール', 'を'): {'words': ['飲も'], 'weights': [2]},
 ('を', '飲も'): {'words': ['う'], 'weights': [2]},
 ('飲も', 'う'): {'words': ['__END__'], 'weights': [2]},
 ('__BEGIN__', 'ビール'): {'words': ['を'], 'weights': [1]},
 ('ビール', 'は'): {'words': ['生'], 'weights': [1]},
 ('は', '生'): {'words': ['__END__'], 'weights': [1]}}